# ENSF 611 Final Project: Weather Temperature Prediction
## Improving Temperature Forecasts Using Machine Learning

**Group Members:** Cameron Dunn, Manuja Senanayake, Edmund Yu, Zohara Kamal

### Project Overview
This project aims to improve upon standard 24-hour advance temperature forecasts by analyzing the discrepancy between forecasted and observed temperatures in Calgary. We'll train and compare three regression models:
- Linear Regression (baseline)
- Support Vector Regressor (SVR)
- Gradient Boosting Regressor

### Objective
Can a machine learning model find trends in the discrepancy between forecasted weather 24hrs in advance and observed temperature such that it can use the forecast to produce more accurate temperature predictions?

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Machine Learning - Models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

# Machine Learning - Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Load Data

In [ ]:
# Load forecast data (24-hour advance predictions)
forecast_df = pd.read_csv('data/raw/forecast.csv', skiprows=2)

# Load observed temperature data
observed_df = pd.read_csv('data/raw/observed.csv', skiprows=2)

print(f"Forecast data shape: {forecast_df.shape}")
print(f"Observed data shape: {observed_df.shape}")
print("\nForecast data columns:")
print(forecast_df.columns.tolist())

## 3. Checking out the data

In [ ]:
# Display first few rows of forecast data
print("Forecast Data:")
display(forecast_df.head())

print("\nObserved Data:")
display(observed_df.head())

# Check data types
print("\nForecast Data Info:")
print(forecast_df.info())

print("\nObserved Data Info:")
print(observed_df.info())

In [ ]:
# Checking for missing values
print("Missing values in forecast data:")
print(forecast_df.isnull().sum())

print("\nMissing values in observed data:")
print(observed_df.isnull().sum())

In [ ]:
# Basic statistics
print("Forecast Data Statistics:")
display(forecast_df.describe())

print("\nObserved Data Statistics:")
display(observed_df.describe())

## 4. Data Cleaning and Preprocessing

In [ ]:
# Convert time column to datetime
forecast_df['time'] = pd.to_datetime(forecast_df['time'])
observed_df['time'] = pd.to_datetime(observed_df['time'])

# Sort by time
forecast_df = forecast_df.sort_values('time').reset_index(drop=True)
observed_df = observed_df.sort_values('time').reset_index(drop=True)

print("Date range of forecast data:")
print(f"From: {forecast_df['time'].min()}")
print(f"To: {forecast_df['time'].max()}")

print("\nDate range of observed data:")
print(f"From: {observed_df['time'].min()}")
print(f"To: {observed_df['time'].max()}")

In [ ]:
# Merge forecast and observed data on time
df = forecast_df.merge(observed_df, on='time', how='inner', suffixes=('_forecast', '_observed'))

In [ ]:
# Remove columns with more than 25% nulls
null_threshold = 0.25
cols_to_drop = df.columns[df.isnull().mean() > null_threshold]
df = df.drop(columns=cols_to_drop)
print(f"Removed columns with >{null_threshold * 100}% nulls: {list(cols_to_drop)}")

In [ ]:
# Drop rows with missing values
df_clean = df.dropna()

## 5. Feature Engineering

### 5.1 Time Encoding (Hour, Day, Month)

In [ ]:
# Extract time components
df_clean['hour'] = df_clean['time'].dt.hour
df_clean['day'] = df_clean['time'].dt.day
df_clean['month'] = df_clean['time'].dt.month
df_clean['day_of_year'] = df_clean['time'].dt.dayofyear
df_clean['day_of_week'] = df_clean['time'].dt.dayofweek

# Cyclical encoding for hour (0-23)
df_clean['hour_sin'] = np.sin(2 * np.pi * df_clean['hour'] / 24)
df_clean['hour_cos'] = np.cos(2 * np.pi * df_clean['hour'] / 24)

# Cyclical encoding for month (1-12)
df_clean['month_sin'] = np.sin(2 * np.pi * df_clean['month'] / 12)
df_clean['month_cos'] = np.cos(2 * np.pi * df_clean['month'] / 12)

print("Time features created")
print("\nNew time-based columns:")
print(['hour', 'day', 'month', 'day_of_year', 'day_of_week', 
       'hour_sin', 'hour_cos', 'month_sin', 'month_cos'])

### 5.2 Wind Direction Encoding (Cosine and Sine)

In [ ]:
# Wind direction is in degrees (0-360)
# Convert to radians and apply sin/cos transformation
wind_col = 'wind_direction_10m_previous_day1 (°)'

if wind_col in df_clean.columns:
    df_clean['wind_dir_sin'] = np.sin(np.radians(df_clean[wind_col]))
    df_clean['wind_dir_cos'] = np.cos(np.radians(df_clean[wind_col]))
else:
    print(f"Warning: {wind_col} not found in dataset")

## 6. Visualization and Exploratory Data Analysis

In [ ]:
# Ensure forecast_error exists for EDA/plots (not used in modeling!)
df_clean['forecast_error'] = df_clean['temperature_2m_previous_day1 (°C)'] - df_clean['temperature_2m (°C)']

# Visualize temperature trends
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot first 30 days
sample_data = df_clean.head(24*30)  # First 30 days

axes[0].plot(sample_data['time'], sample_data['temperature_2m_previous_day1 (°C)'], 
             label='Forecast', alpha=0.7, linewidth=1)
axes[0].plot(sample_data['time'], sample_data['temperature_2m (°C)'], 
             label='Observed', alpha=0.7, linewidth=1)
axes[0].set_title('Forecast vs Observed Temperature (First 30 Days)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Temperature (°C)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot forecast error
axes[1].plot(sample_data['time'], sample_data['forecast_error'], 
             color='red', alpha=0.6, linewidth=1)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].set_title('Forecast Error (Forecast - Observed)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Error (°C)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Distribution of forecast error
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df_clean['forecast_error'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df_clean['forecast_error'].mean(), color='red', 
                linestyle='--', linewidth=2, label=f"Mean: {df_clean['forecast_error'].mean():.2f}°C")
axes[0].set_title('Distribution of Forecast Error', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Error (°C)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot by month
df_clean.boxplot(column='forecast_error', by='month', ax=axes[1])
axes[1].set_title('Forecast Error by Month', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Error (°C)')
plt.suptitle('')  # Remove the default title

plt.tight_layout()
plt.show()

print(f"Mean Absolute Forecast Error: {abs(df_clean['forecast_error']).mean():.2f}°C")
print(f"RMSE of raw forecast: {np.sqrt(mean_squared_error(df_clean['temperature_2m (°C)'], df_clean['temperature_2m_previous_day1 (°C)'])):.2f}°C")

# Correlation heatmap
# Select numeric columns for correlation
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
correlation_cols = [col for col in numeric_cols if col not in ['time']]

# Calculate correlation with target variable
target_corr = df_clean[correlation_cols].corrwith(df_clean['temperature_2m (°C)']).sort_values(ascending=False)

plt.figure(figsize=(10, 8))
target_corr.plot(kind='barh')
plt.title('Correlation with Observed Temperature', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.tight_layout()
plt.show()

print("\nTop 10 features correlated with observed temperature:")
print(target_corr.head(10))

## 7. Prepare Data for Modeling / Pipeline

In [ ]:
# Use only the forecast as feature, and observed as target
feature_col = 'temperature_2m_previous_day1 (°C)'
target_col = 'temperature_2m (°C)'

X = df_clean[[feature_col]].copy()
y = df_clean[target_col].copy()


In [ ]:
# Split data into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 8. Model Training

### 8.1 Baseline Model: Linear Regression

In [ ]:
print("LINEAR REGRESSION")

lr_param_grid = {}

lr_grid = GridSearchCV(
    LinearRegression(),
    lr_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

lr_grid.fit(X_train_scaled, y_train)

### 8.2 Support Vector Regressor

In [ ]:
print("SUPPORT VECTOR REGRESSOR")

svr_param_grid = {
    "svr__kernel": ["rbf", "poly"],
    "svr__C": [0.1, 1, 10, 50],
    "svr__epsilon": [0.01, 0.1, 0.5],
    "svr__gamma": ["scale", "auto"]
}

svr_grid = GridSearchCV(
    SVR(),
    svr_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

svr_grid.fit(X_train_scaled, y_train)

### 8.3 Gradient Boosting Regressor

In [ ]:
print("GRADIENT BOOSTING REGRESSOR")

gbr_param_grid = {
    "gbr__n_estimators": [100, 200, 400],
    "gbr__learning_rate": [0.01, 0.05, 0.1],
    "gbr__max_depth": [2, 3, 4],
    "gbr__subsample": [0.7, 1.0]
}

gbr_grid = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    gbr_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

gbr_grid.fit(X_train_scaled, y_train)

## 9. Model Comparison and Evaluation

In [ ]:
# Compile results
results_df = pd.DataFrame([
    lr_results,
    svr_results,
    svr_best_results,
    gb_results,
    gb_best_results
]).drop(columns=['predictions'])

print("\n" + "=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
display(results_df)

# Find best model
best_model_idx = results_df['Test MAE'].idxmin()
best_model_name = results_df.loc[best_model_idx, 'Model']
print(f"\n Best Model: {best_model_name}")
print(f"   Test MAE:  {results_df.loc[best_model_idx, 'Test MAE']:.4f}°C")
print(f"   Test RMSE: {results_df.loc[best_model_idx, 'Test RMSE']:.4f}°C")t

In [ ]:
# R² Score comparison
fig, ax = plt.subplots(figsize=(12, 6))

x_pos = np.arange(len(results_df))
ax.bar(x_pos - 0.2, results_df['Train R²'], 0.4, label='Train R²', alpha=0.8)
ax.bar(x_pos + 0.2, results_df['Test R²'], 0.4, label='Test R²', alpha=0.8)
ax.set_xlabel('Model')
ax.set_ylabel('R² Score')
ax.set_title('R² Score Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
# ax.set_ylim([0.95, 1.0])  # Zoom in to see differences

plt.tight_layout()
plt.show()

In [ ]:
# Calculate and display how much the best model improves forecast accuracy
# compared to the original raw forecast. This compares the RMSE of the raw forecast
# (using the forecasted temperature directly) to the RMSE of the best model
# on the test set, and reports both the absolute and percent improvement.

raw_rmse = np.sqrt(mean_squared_error(df_clean['temperature_2m (°C)'], df_clean['temperature_2m_previous_day1 (°C)']))
best_model_rmse = results_df.loc[best_model_idx, 'Test RMSE']
best_model_name = results_df.loc[best_model_idx, 'Model']
improvement = raw_rmse - best_model_rmse
percent_improvement = (improvement / raw_rmse) * 100

print(f"Raw forecast RMSE: {raw_rmse:.2f}°C")
print(f"{best_model_name} RMSE: {best_model_rmse:.2f}°C")
print(f"Absolute improvement: {improvement:.2f}°C")
print(f"Percent improvement: {percent_improvement:.1f}%")

## 10. Prediction Visualization

In [ ]:
# After dropping NaNs, reset index to ensure alignment
df_clean = df_clean.reset_index(drop=True)


# When plotting predictions, use a direct range for the test set
sample_size = min(24 * 7, len(y_test))
sample_indices = range(sample_size)

# Get timestamps for test set (last len(y_test) rows after reset_index)
test_times = df_clean['time'].iloc[-len(y_test):].values

plt.figure(figsize=(16, 8))

plt.plot(test_times[sample_indices], y_test.values[sample_indices],
         label='Actual Temperature', linewidth=2, color='black', marker='o', markersize=3)

plt.plot(test_times[sample_indices], lr_results['predictions'][sample_indices],
         label='Linear Regression', alpha=0.7, linewidth=1.5)

plt.plot(test_times[sample_indices], svr_best_results['predictions'][sample_indices],
         label='SVR (tuned)', alpha=0.7, linewidth=1.5)

plt.plot(test_times[sample_indices], gb_best_results['predictions'][sample_indices],
         label='Gradient Boosting (tuned)', alpha=0.7, linewidth=1.5)

plt.title('Model Predictions vs Actual Temperature (First Week of Test Set)',
          fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Residual plots for best models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_to_plot = [
    ('Linear Regression', lr_results['predictions']),
    ('SVR (tuned)', svr_best_results['predictions']),
    ('Gradient Boosting (tuned)', gb_best_results['predictions'])
]

for idx, (name, predictions) in enumerate(models_to_plot):
    residuals = y_test.values - predictions
    
    axes[idx].scatter(predictions, residuals, alpha=0.5, s=10)
    axes[idx].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[idx].set_xlabel('Predicted Temperature (°C)')
    axes[idx].set_ylabel('Residuals (°C)')
    axes[idx].set_title(f'{name}\nResidual Plot', fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Error distribution comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, predictions) in enumerate(models_to_plot):
    errors = np.abs(y_test.values - predictions)
    
    axes[idx].hist(errors, bins=50, edgecolor='black', alpha=0.7)
    axes[idx].axvline(errors.mean(), color='red', linestyle='--', 
                     linewidth=2, label=f'Mean: {errors.mean():.3f}°C')
    axes[idx].set_xlabel('Absolute Error (°C)')
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{name}\nError Distribution', fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 11. Feature Importance Analysis

In [ ]:
# Feature importance for best model
if best_model_name.startswith('Gradient Boosting'):
    best_estimator = gb_grid.best_estimator_
    importances = best_estimator.feature_importances_
    importance_label = 'Feature Importance'
elif best_model_name.startswith('Linear Regression'):
    best_estimator = lr_model
    importances = np.abs(best_estimator.coef_)
    importance_label = 'Absolute Coefficient'
elif best_model_name.startswith('SVR'):
    print("Feature importance is not directly available for SVR models.")
    importances = None

if importances is not None:
    feature_importance = pd.DataFrame({
        'feature': [feature_col],
        'importance': importances
    }).sort_values('importance', ascending=False)

    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    plt.barh(range(20), feature_importance['importance'].head(20))
    plt.yticks(range(20), feature_importance['feature'].head(20))
    plt.xlabel(importance_label)
    plt.title(f'Top 20 Most Important Features ({best_model_name})', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    print("\nTop 10 Most Important Features:")
    display(feature_importance.head(10))

In [ ]:
# Plot: Absolute error of raw forecast vs best model on test set (auto-select best model)

# Get test set indices (last len(y_test) rows after reset_index)
test_times = df_clean['time'].iloc[-len(y_test):].values
raw_forecast = df_clean['temperature_2m_previous_day1 (°C)'].iloc[-len(y_test):].values

# Map model names to their predictions
model_predictions = {
    'Linear Regression': lr_results['predictions'],
    'SVR (default)': svr_results['predictions'],
    'SVR (tuned)': svr_best_results['predictions'],
    'Gradient Boosting (default)': gb_results['predictions'],
    'Gradient Boosting (tuned)': gb_best_results['predictions'],
}

# Compute absolute errors
raw_error = np.abs(raw_forecast - y_test.values)
best_model_error = np.abs(model_predictions[best_model_name] - y_test.values)

# Compute the difference in absolute error
error_difference = raw_error - best_model_error

fig, ax = plt.subplots(figsize=(15, 4))

ax.plot(test_times[sample_indices], error_difference[sample_indices],
    color='blue', alpha=0.6, linewidth=1)
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.set_title(f'Error Difference: Raw Forecast Error - {best_model_name} Error',
         fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Error Difference')
ax.grid(True, alpha=0.3)

# Highlight where best model improves (positive = raw worse than model)
ax.fill_between(test_times[sample_indices], error_difference[sample_indices], 0,
        where=error_difference[sample_indices] >= 0,
        facecolor='green', alpha=0.2, interpolate=True)
ax.fill_between(test_times[sample_indices], error_difference[sample_indices], 0,
        where=error_difference[sample_indices] < 0,
        facecolor='red', alpha=0.2, interpolate=True)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## 12. Conclusions and Summary

After testing out linear regression, SVR, and gradient boosting for predicting the next day’s temperature using just the forecasted value, we found that the default SVR model actually worked best on our test data. The forecast itself turned out to be a strong predictor, but we noticed it often underestimated the actual temperature. By applying a machine learning model, we managed to bring the RMSE down from 2.23°C (just using the raw forecast) to 1.82°C with SVR. That’s an improvement of about 0.41°C, or roughly 18%. Overall, even with just one main feature, these simple models did a pretty good job, and it shows that correcting for systematic bias in forecasts can make a noticeable difference.